In [1]:
import os
import sys
import pandas as pd
import numpy as np
import PIL

seed = 16
np.random.seed(seed)

from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator


c:\users\garrick\anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#check using system GPU for processing and declaring system/GPU parameters

from tensorflow.python.client import device_lib
import tensorflow as tf
os.environ["CUDA_VISIBLE_DEVICES"]="0" #for training on gpu
print(device_lib.list_local_devices())

# configure tensorflow before fitting model
tf_config = tf.ConfigProto()
tf_config.gpu_options.per_process_gpu_memory_fraction = 0.99
sess = tf.Session(config=tf_config)


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17132357235513740450
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 1493781708
locality {
  bus_id: 1
}
incarnation: 18400566845976949322
physical_device_desc: "device: 0, name: GeForce GTX 860M, pci bus id: 0000:01:00.0, compute capability: 5.0"
]


In [3]:
# declare ImageDataGenerator and flow_from_directory vars
batch_size=10
num_classes = 120

In [4]:
# changing directory for flow_from_directory method
os.chdir('C:\\Users\\Garrick\\Documents\\Springboard\\Capstone Project 2\\datasets')

In [5]:
#obtain train and test labels
from scipy.io import loadmat

#y_train_rough = loadmat(r'''C:\\Users\\Garrick\\Documents\\Springboard\\Capstone Project 2\\datasets\\train_list.mat''')['labels']

#y_test = loadmat(r'''C:\\Users\\Garrick\\Documents\\Springboard\\Capstone Project 2\\datasets\\test_list.mat''')['labels']

In [6]:
from sklearn.model_selection import train_test_split

In [21]:
labels = loadmat(r'''C:\\Users\\Garrick\\Documents\\Springboard\\Capstone Project 2\\datasets\\train_list.mat''')['labels']
files = loadmat(r'''C:\\Users\\Garrick\\Documents\\Springboard\\Capstone Project 2\\datasets\\train_list.mat''')['file_list']
labels = [item for label in labels for item in label] #this is flattening a list of lists, because for some reason ever label is stored as a list
files = [item for file in files for item in file]
df = pd.DataFrame({'labels':labels, 'files':files})
train, validate = train_test_split(df, test_size = 0.2, stratify=labels)


In [22]:
X_train = train['files']
y_train = to_categorical(train['labels'])
X_val = validate['files']
y_val = to_categorical(validate['labels'])

In [5]:
train_datagen = ImageDataGenerator(rotation_range=15, shear_range=0.1, channel_shift_range=20,
                                    width_shift_range=0.1,  height_shift_range=0.1, zoom_range=0.1, horizontal_flip=True,
                                    fill_mode='nearest', rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)



train_generator = train_datagen.flow_from_directory('train', target_size=(224,224), color_mode='rgb',
            class_mode='categorical', shuffle=False, batch_size=25)

validation_generator = validation_datagen.flow_from_directory('validation', target_size=(224,224), color_mode='rgb',
            class_mode='categorical', shuffle=False, batch_size=25)


test_generator = test_datagen.flow_from_directory('test', target_size=(224,224), color_mode='rgb',
            class_mode='categorical', shuffle=False, batch_size=25)


Found 9600 images belonging to 120 classes.
Found 2400 images belonging to 120 classes.
Found 8580 images belonging to 120 classes.


In [24]:
#obtain validation labels
# no longer used, utilizing sklearn's train_test_split function

'''
y_validation = []
y_train = []

for i in range(120):
    i = i*100
    
    #begin_index = train_full_list[i::100]
    #end_index = train_full_list[i+100::100]
    
    slice = y_train_rough[i:i+100,]
    y_train.append(slice[:80])
    y_validation.append(slice[80:])

y_train = np.concatenate(y_train, axis=0)
y_validation = np.concatenate(y_validation, axis=0)
'''

'\ny_validation = []\ny_train = []\n\nfor i in range(120):\n    i = i*100\n    \n    #begin_index = train_full_list[i::100]\n    #end_index = train_full_list[i+100::100]\n    \n    slice = y_train_rough[i:i+100,]\n    y_train.append(slice[:80])\n    y_validation.append(slice[80:])\n\ny_train = np.concatenate(y_train, axis=0)\ny_validation = np.concatenate(y_validation, axis=0)\n'

In [6]:
# using a simple CNN to start

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.optimizers import Adam
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
import keras.utils
from keras import backend as K
K.set_image_dim_ordering('tf')

from keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=2)

In [7]:
input_shape = (224,224, 3)

# create the model

base_model = Sequential()
base_model.add(Conv2D(64, (11, 11), strides=4, input_shape=input_shape, padding='valid', activation='relu', kernel_constraint=maxnorm(3)))
base_model.add(MaxPooling2D(pool_size=(2, 2)))

base_model.add(Conv2D(64, (4, 4), strides=2, activation='relu', padding='valid', kernel_constraint=maxnorm(3)))
base_model.add(MaxPooling2D(pool_size=(2, 2)))

base_model.add(Conv2D(64, (4, 4), strides=2, activation='relu', padding='valid', kernel_constraint=maxnorm(3)))
base_model.add(Flatten())

base_model.add(Dense(256, activation='relu', kernel_constraint=maxnorm(3)))
base_model.add(Dropout(0.2))
base_model.add(Dense(num_classes, activation='softmax'))

# Compile model
adam_op = Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
base_model.compile(loss='categorical_crossentropy', optimizer=adam_op, metrics=['accuracy'])
print(base_model.summary())


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 54, 54, 64)        23296     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 27, 27, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 64)        65600     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 2, 2, 64)          65600     
_________________________________________________________________
flatten_1 (Flatten)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               65792     
__________

In [13]:
# train/fit the simple CNN using flow from directory 

base_model.fit_generator(train_generator, validation_data=validation_generator,
                         steps_per_epoch=300, epochs=25, callbacks=[early_stopping])

Epoch 1/25
300/300 [==============================] - 158s 526ms/step - loss: 4.7910 - acc: 0.0075 - val_loss: 4.7881 - val_acc: 0.0079
Epoch 2/25
300/300 [==============================] - 156s 519ms/step - loss: 4.7896 - acc: 0.0088 - val_loss: 4.7874 - val_acc: 0.0096
Epoch 3/25
300/300 [==============================] - 152s 507ms/step - loss: 4.7915 - acc: 0.0088 - val_loss: 4.7869 - val_acc: 0.0083
Epoch 4/25
300/300 [==============================] - 153s 508ms/step - loss: 4.7880 - acc: 0.0088 - val_loss: 4.7870 - val_acc: 0.0083
Epoch 5/25
300/300 [==============================] - 154s 513ms/step - loss: 4.7886 - acc: 0.0083 - val_loss: 4.7866 - val_acc: 0.0071
Epoch 6/25
300/300 [==============================] - 158s 526ms/step - loss: 4.7872 - acc: 0.0089 - val_loss: 4.7862 - val_acc: 0.0096
Epoch 7/25
300/300 [==============================] - 155s 516ms/step - loss: 4.7890 - acc: 0.0092 - val_loss: 4.7861 - val_acc: 0.0096
Epoch 8/25
300/300 [============================

In [14]:
base_model.save('base_model.h5')

In [15]:
# tweak base model... decrease first covnet filter, decease filter size and stride

base_model_v2 = Sequential()
base_model_v2.add(Conv2D(32, (3, 3), strides=1, input_shape=input_shape, padding='valid', activation='relu', kernel_constraint=maxnorm(3)))
base_model_v2.add(MaxPooling2D(pool_size=(2, 2)))

base_model_v2.add(Conv2D(32, (3, 3), strides=1, activation='relu', padding='valid', kernel_constraint=maxnorm(3)))
base_model_v2.add(MaxPooling2D(pool_size=(2, 2)))

base_model_v2.add(Conv2D(32, (4, 4), strides=2, activation='relu', padding='valid', kernel_constraint=maxnorm(3)))
base_model_v2.add(MaxPooling2D(pool_size=(2, 2)))
base_model_v2.add(Flatten())

base_model_v2.add(Dense(256, activation='relu', kernel_constraint=maxnorm(3)))
base_model_v2.add(Dropout(0.2))
base_model_v2.add(Dense(num_classes, activation='softmax'))

# Compile model
adam_op = Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
base_model_v2.compile(loss='categorical_crossentropy', optimizer=adam_op, metrics=['accuracy'])
print(base_model_v2.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 222, 222, 32)      896       
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 111, 111, 32)      0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 109, 109, 32)      9248      
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 54, 54, 32)        0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 26, 26, 32)        16416     
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 13, 13, 32)        0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 5408)              0         
__________

In [16]:
base_model_v2.fit_generator(train_generator, validation_data=validation_generator,
                         steps_per_epoch=300, epochs=10, callbacks=[early_stopping])

Epoch 1/10
 32/300 [==>...........................] - ETA: 3:28 - loss: 4.7879 - acc: 0.0012

KeyboardInterrupt: 

In [ ]:
# trying a larger FC layer

base_model_v3 = Sequential()
base_model_v3.add(Conv2D(64, (11, 11), strides=4, input_shape=input_shape, padding='valid', activation='relu', kernel_constraint=maxnorm(3)))
base_model_v3.add(MaxPooling2D(pool_size=(2, 2)))

base_model_v3.add(Conv2D(32, (4, 4), strides=2, activation='relu', padding='valid', kernel_constraint=maxnorm(3)))
base_model_v3.add(MaxPooling2D(pool_size=(2, 2)))

base_model_v3.add(Conv2D(64, (4, 4), strides=2, activation='relu', padding='valid', kernel_constraint=maxnorm(3)))
base_model_v3.add(Flatten())

base_model_v3.add(Dense(512, activation='relu', kernel_constraint=maxnorm(3)))
base_model_v3.add(Dropout(0.2))
base_model_v3.add(Dense(num_classes, activation='softmax'))

# Compile model
adam_op = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
base_model_v3.compile(loss='categorical_crossentropy', optimizer=adam_op, metrics=['accuracy'])
print(base_model_v3.summary())

In [11]:
# trying deep_model architecture (in this case, adding additional conv layer and pooling after 2 conv +activation layers

deep_model = Sequential()

deep_model = Sequential()
deep_model.add(Conv2D(96, (11, 11), strides=4, input_shape=input_shape, padding='valid', activation='relu', kernel_constraint=maxnorm(3)))
deep_model.add(MaxPooling2D(pool_size=(2, 2)))

deep_model.add(Conv2D(64, (3, 3), strides=1, activation='relu', padding='valid', kernel_constraint=maxnorm(3)))
deep_model.add(Conv2D(64, (3, 3), strides=1, activation='relu', padding='valid', kernel_constraint=maxnorm(3)))
deep_model.add(MaxPooling2D(pool_size=(2, 2)))

deep_model.add(Conv2D(32, (3, 3), strides=1, activation='relu', padding='valid', kernel_constraint=maxnorm(3)))
deep_model.add(Conv2D(32, (3, 3), strides=1, activation='relu', padding='valid', kernel_constraint=maxnorm(3)))
deep_model.add(MaxPooling2D(pool_size=(2, 2)))

deep_model.add(Flatten())

deep_model.add(Dense(256, activation='relu', kernel_constraint=maxnorm(3)))
deep_model.add(Dropout(0.2))
deep_model.add(Dense(num_classes, activation='softmax'))
    
# Compile model
# use existing adam optimizer
deep_model.compile(loss='categorical_crossentropy', optimizer=adam_op, metrics=['accuracy'])
print(deep_model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_24 (Conv2D)           (None, 54, 54, 96)        34944     
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 27, 27, 96)        0         
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 25, 25, 64)        55360     
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 23, 23, 64)        36928     
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 11, 11, 64)        0         
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 9, 9, 32)          18464     
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 7, 7, 32)          9248      
__________

In [12]:
deep_model.fit_generator(train_generator, validation_data=validation_generator,
                         steps_per_epoch=300, epochs=50, callbacks=[early_stopping])

# not enough steps per epoch

Epoch 1/50
300/300 [==============================] - 216s 720ms/step - loss: 4.7901 - acc: 0.0076 - val_loss: 4.7877 - val_acc: 0.0071
Epoch 2/50
300/300 [==============================] - 156s 520ms/step - loss: 4.7890 - acc: 0.0063 - val_loss: 4.7873 - val_acc: 0.0063
Epoch 3/50
300/300 [==============================] - 150s 499ms/step - loss: 4.7874 - acc: 0.0072 - val_loss: 4.7871 - val_acc: 0.0121
Epoch 4/50
300/300 [==============================] - 155s 517ms/step - loss: 4.7895 - acc: 0.0064 - val_loss: 4.7872 - val_acc: 0.0071
Epoch 5/50
300/300 [==============================] - 156s 520ms/step - loss: 4.7877 - acc: 0.0085 - val_loss: 4.7871 - val_acc: 0.0079
Epoch 6/50
300/300 [==============================] - 153s 509ms/step - loss: 4.7875 - acc: 0.0091 - val_loss: 4.7870 - val_acc: 0.0088
Epoch 7/50
300/300 [==============================] - 156s 519ms/step - loss: 4.7879 - acc: 0.0087 - val_loss: 4.7868 - val_acc: 0.0092
Epoch 8/50
300/300 [============================

In [14]:
deep_model.fit_generator(train_generator, validation_data=validation_generator,
                         steps_per_epoch=800, epochs=25, callbacks=[early_stopping])

Epoch 1/25
800/800 [==============================] - 410s 512ms/step - loss: 4.7871 - acc: 0.0096 - val_loss: 4.7863 - val_acc: 0.0100
Epoch 2/25
800/800 [==============================] - 422s 527ms/step - loss: 4.7871 - acc: 0.0090 - val_loss: 4.7855 - val_acc: 0.0125
Epoch 3/25
800/800 [==============================] - 419s 524ms/step - loss: 4.7865 - acc: 0.0082 - val_loss: 4.7846 - val_acc: 0.0108
Epoch 4/25
800/800 [==============================] - 407s 509ms/step - loss: 4.7850 - acc: 0.0117 - val_loss: 4.7827 - val_acc: 0.0087
Epoch 5/25
800/800 [==============================] - 405s 506ms/step - loss: 4.7836 - acc: 0.0097 - val_loss: 4.7805 - val_acc: 0.0112
Epoch 6/25
800/800 [==============================] - 403s 504ms/step - loss: 4.7829 - acc: 0.0099 - val_loss: 4.7776 - val_acc: 0.0100
Epoch 7/25
800/800 [==============================] - 412s 515ms/step - loss: 4.7767 - acc: 0.0127 - val_loss: 4.7714 - val_acc: 0.0104
Epoch 8/25
800/800 [============================

In [15]:
deep_model.save('deep_model.h5')

In [52]:
# old iteration

model_3 = Sequential()
model_3.add(Conv2D(64, (3, 3), strides=1, input_shape=input_shape, padding='valid', activation='relu', kernel_constraint=maxnorm(3)))
model_3.add(Dropout(0.2))
model_3.add(MaxPooling2D(pool_size=(2, 2)))

model_3.add(Conv2D(32, (3, 3), strides=1, activation='relu', padding='valid', kernel_constraint=maxnorm(3)))
model_3.add(Dropout(0.2))
model_3.add(MaxPooling2D(pool_size=(2, 2)))

model_3.add(Conv2D(32, (3, 3), strides=1, activation='relu', padding='valid', kernel_constraint=maxnorm(3)))
model_3.add(Dropout(0.2))
model_3.add(MaxPooling2D(pool_size=(3, 3)))
model_3.add(Flatten())

model_3.add(Dense(256, activation='relu', kernel_constraint=maxnorm(3)))
model_3.add(Dropout(0.2))
model_3.add(Dense(num_classes, activation='softmax'))
     

# Compile model
model_3.compile(loss='categorical_crossentropy', optimizer=adam_op, metrics=['accuracy'])
print(model_3.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_89 (Conv2D)           (None, 222, 222, 64)      1792      
_________________________________________________________________
dropout_51 (Dropout)         (None, 222, 222, 64)      0         
_________________________________________________________________
max_pooling2d_73 (MaxPooling (None, 111, 111, 64)      0         
_________________________________________________________________
conv2d_90 (Conv2D)           (None, 109, 109, 32)      18464     
_________________________________________________________________
dropout_52 (Dropout)         (None, 109, 109, 32)      0         
_________________________________________________________________
max_pooling2d_74 (MaxPooling (None, 54, 54, 32)        0         
_________________________________________________________________
conv2d_91 (Conv2D)           (None, 52, 52, 32)        9248      
__________

In [54]:
# doesn't work
#model_3.fit_generator(train_generator, validation_data=validation_generator,
#                    steps_per_epoch=300, epochs=25, callbacks=[early_stopping])

In [8]:
wide_model_slow_learn = Sequential()
wide_model_slow_learn.add(Conv2D(32, (3, 3), strides=1, input_shape=input_shape, padding='valid', activation='relu', kernel_constraint=maxnorm(3)))
wide_model_slow_learn.add(MaxPooling2D(pool_size=(2, 2)))

wide_model_slow_learn.add(Conv2D(32, (3, 3), strides=1, activation='relu', padding='valid', kernel_constraint=maxnorm(3)))
wide_model_slow_learn.add(MaxPooling2D(pool_size=(2, 2)))

wide_model_slow_learn.add(Conv2D(32, (3, 3), strides=2, activation='relu', padding='valid', kernel_constraint=maxnorm(3)))
wide_model_slow_learn.add(MaxPooling2D(pool_size=(2, 2)))
wide_model_slow_learn.add(Flatten())

wide_model_slow_learn.add(Dense(256, activation='relu', kernel_constraint=maxnorm(3)))
wide_model_slow_learn.add(Dropout(0.2))
wide_model_slow_learn.add(Dense(num_classes, activation='softmax'))
    
# Compile model

adam_op = Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
wide_model_slow_learn.compile(loss='categorical_crossentropy', optimizer=adam_op, metrics=['accuracy'])
print(wide_model_slow_learn.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 222, 222, 32)      896       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 111, 111, 32)      0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 109, 109, 32)      9248      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 54, 54, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 26, 26, 32)        9248      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 5408)              0         
__________

In [9]:
wide_model_slow_learn.fit_generator(train_generator, validation_data=validation_generator,
                         steps_per_epoch=300, epochs=50, callbacks=[early_stopping])

Epoch 1/50
300/300 [==============================] - 162s 540ms/step - loss: 4.8012 - acc: 0.0101 - val_loss: 4.7898 - val_acc: 0.0079
Epoch 2/50
300/300 [==============================] - 155s 516ms/step - loss: 4.7940 - acc: 0.0045 - val_loss: 4.7881 - val_acc: 0.0083
Epoch 3/50
300/300 [==============================] - 154s 512ms/step - loss: 4.7945 - acc: 0.0117 - val_loss: 4.7869 - val_acc: 0.0079
Epoch 4/50
300/300 [==============================] - 155s 517ms/step - loss: 4.7954 - acc: 0.0056 - val_loss: 4.7859 - val_acc: 0.0087
Epoch 5/50
300/300 [==============================] - 159s 528ms/step - loss: 4.7907 - acc: 0.0077 - val_loss: 4.7855 - val_acc: 0.0096
Epoch 6/50
300/300 [==============================] - 159s 530ms/step - loss: 4.7907 - acc: 0.0093 - val_loss: 4.7851 - val_acc: 0.0079
Epoch 7/50
300/300 [==============================] - 155s 517ms/step - loss: 4.7896 - acc: 0.0111 - val_loss: 4.7849 - val_acc: 0.0088
Epoch 8/50
300/300 [============================

KeyboardInterrupt: 

In [10]:
wide_model_fast_learn = Sequential()
wide_model_fast_learn.add(Conv2D(32, (3, 3), strides=1, input_shape=input_shape, padding='valid', activation='relu', kernel_constraint=maxnorm(3)))
wide_model_fast_learn.add(MaxPooling2D(pool_size=(2, 2)))

wide_model_fast_learn.add(Conv2D(32, (3, 3), strides=1, activation='relu', padding='valid', kernel_constraint=maxnorm(3)))
wide_model_fast_learn.add(MaxPooling2D(pool_size=(2, 2)))

wide_model_fast_learn.add(Conv2D(32, (3, 3), strides=2, activation='relu', padding='valid', kernel_constraint=maxnorm(3)))
wide_model_fast_learn.add(MaxPooling2D(pool_size=(2, 2)))
wide_model_fast_learn.add(Flatten())

wide_model_fast_learn.add(Dense(256, activation='relu', kernel_constraint=maxnorm(3)))
wide_model_fast_learn.add(Dropout(0.2))
wide_model_fast_learn.add(Dense(num_classes, activation='softmax'))
    
# Compile model

adam_op = Adam(lr=1, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
wide_model_fast_learn.compile(loss='categorical_crossentropy', optimizer=adam_op, metrics=['accuracy'])
print(wide_model_fast_learn.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 222, 222, 32)      896       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 111, 111, 32)      0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 109, 109, 32)      9248      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 54, 54, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 26, 26, 32)        9248      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 5408)              0         
__________

In [11]:
wide_model_fast_learn.fit_generator(train_generator, validation_data=validation_generator,
                         steps_per_epoch=300, epochs=50, callbacks=[early_stopping])

Epoch 1/50
300/300 [==============================] - 162s 541ms/step - loss: 15.9624 - acc: 0.0073 - val_loss: 15.9838 - val_acc: 0.0083
Epoch 2/50
300/300 [==============================] - 156s 519ms/step - loss: 15.9999 - acc: 0.0073 - val_loss: 15.9838 - val_acc: 0.0083
Epoch 3/50
300/300 [==============================] - 152s 506ms/step - loss: 16.0106 - acc: 0.0067 - val_loss: 15.9838 - val_acc: 0.0083


In [ ]:
ImageDataGenerator?